# Generate exercise precursors
This notebook generates sentence pairs ready for quality control to incorporate into pre-generated language learning exercises.

**Inputs:** See below
- Most important is the `IN_WORDS` parameter which is a list of words (in target language) for which exercises should be generated.
  The list is loaded from the `data-import/in-words.tsv` file which is expected to have the following structure:
  ```
  word
  Haus
  Maus
  ...
  ```

**Outputs:** A list of exercise precursors stored in `data-generated/exercise-precursors.tsv`

This list can be quality checked, before it will be finally used to populate the exercise table which is exposed in the API.

**Prerequisites:**

- Generate and import SQL data (see README).


In [ ]:
%pip install pandas textdistance tqdm

In [ ]:
import gc
import random
import re

import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# Source language of the translation exercices.
IN_SOURCE_LANG = "UK"

# Target language of the translation exercises.
IN_TARGET_LANG = "DE"

# Words in the target language to train. If empty, a few of the most common words are automatically selected for demonstration purposes.
# IN_WORDS = []
IN_WORDS = pd.read_csv("data-import/in-words.tsv", sep="\t").word

# For each word, the number of translation pairs to generate.
IN_NUM_TRANSLATIONS = 5

# Minimum number of words in the target sentence.
IN_MIN_WORDS = 4

# Maximum number of words in the target sentence.
IN_MAX_WORDS = 9

# If IN_WORDS is blank, populate it with this many sample words
DEFAULT_N_WORDS = 10

# If IN_WORDS is blank, find default words having at least this length.
DEFAULT_MIN_WORDS_SIZE = 4

## Use sample data for input words
If `IN_WORDS` is blank, provide some common sample words instead.

In [ ]:
if len(IN_WORDS) == 0:
    def get_words_by_frequency():
        r_nonword = re.compile(r"""[.,?!:;()"]""")
        r_whitespace = re.compile(r"[/\s]+")
        word_counts = {}

        df_sentences = pd.read_csv("./data-generated/sentences.tsv",
                                   sep="\t", index_col="id")
        df_targets = df_sentences[df_sentences.language == IN_TARGET_LANG]

        # Split words and get word counts
        for sentence in tqdm(df_targets.text):
            words = [r_nonword.sub("",w.lower())
                     for w in r_whitespace.split(sentence)]
            for word in words:
                if len(word) >= DEFAULT_MIN_WORDS_SIZE:
                    word_counts[word] = word_counts.get(word, 0) + 1

        sort_dict = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
        sort_dict = dict(sort_dict)

        return sort_dict

    most_common_words = list(get_words_by_frequency().keys())[0:DEFAULT_N_WORDS * 10]
    sampled_common_words = random.sample(most_common_words, DEFAULT_N_WORDS)
    IN_WORDS = sampled_common_words

gc.collect()
IN_WORDS

# Query

In [ ]:
import sqlite3


translations_q = """
SELECT
    word,
    t.id as translation_id,
    source.id as sourceId,
    source.text as sourceText,
    source.language as sourceLanguage,
    target.id as targetId,
    target.text as targetText,
    target.word_count as twc,
    target.language as targetLanguage FROM vocabulary v
                                               JOIN sentence_vocabulary sv on v.id = sv.vocabulary_id
                                               JOIN sentences target on sv.sentence_id = target.id
                                               JOIN translations t on target.id = t.s1 or target.id = t.s2
                                               JOIN sentences source on source.id = t.s1 or source.id = t.s2
WHERE v.language = :target_lang and word = :word and :min_words <= twc and twc <= :max_words
"""

with sqlite3.connect("../taskpool.db") as conn:
    all_results = []
    for word in tqdm(IN_WORDS):
        results = conn.execute(translations_q, {"target_lang": IN_TARGET_LANG, "word": word, "min_words": IN_MIN_WORDS, "max_words": IN_MAX_WORDS}).fetchall()

        added = 0

        for x in results:
            if x[4] == IN_SOURCE_LANG:
                added = added + 1
                all_results.append({
                    "translation_id": x[1],
                    "word": word,
                    "source_sentence_id": x[2],
                    "source_sentence": x[3],
                    "target_language": IN_TARGET_LANG,
                    "target_sentence_id": x[5],
                    "target_sentence": x[6],
                    "difficulty": 0,
                })

                if added >= IN_NUM_TRANSLATIONS:
                    break

        print(len(all_results))

df = pd.DataFrame(all_results)
df.to_csv("data-generated/exercise-precursors.tsv", sep="\t", index=False)
df
